In [1]:
import os
import glob
import time

import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import metpy.calc as mpcalc
from metpy.units import units

# Functions

In [2]:
ERA5M_PATH  = "/global/homes/w/wboos/m3310project/wboos/era5monthlyQuentin/"
ERA5M_PATH2 = "/global/project/projectdirs/m3310/wboos/era5monthly/"

boxNH = [[-100, -70, 12 , 20],
         [-80 , -60, 8  , 16],
         [70  , 90 , 2  , 18],
         [100 , 120, 8  , 20],
         [118 , 140, 4  , 20],
         [100 , 110, 0  , 12],
        ]

boxSH = [[20  , 50 , -20, 0 ],
         [-60 , -30, -20, 0 ],
        ]

boxNH1 = [[b[0]%360,b[1]%360,b[2],b[3]] for b in boxNH]
boxSH1 = [[b[0]%360,b[1]%360,b[2],b[3]] for b in boxSH]

namesNH = ["Central America",
           "Venezuela",
           "South India / Sri Lanka",
           "Vietnam / South China Sea",
           "Philippines",
           "Malaysia"
          ]
namesSH = ["Tanzania",
           "NorthEast Brazil"
          ]

In [3]:
#Data extraction from ERA5
era5yrs = list(range(1979,2019))

def retrieve_era5(year,varid):
    """gather an ERA5 monthly mean variable for the year 'year'
    varid gives the id of the variable in era5
        - year : str, "YYYY"
        - varid : str, eg. "128_130_t" for temperature
    """
    era5var = xr.open_dataset(glob.glob(os.path.join(ERA5M_PATH,"*/e5.*.%s.*.%s*.nc"%(varid,year)))[0])
    varname = list(era5var.data_vars)[0] #get name of the main variable, eg 'T' for temperature
    
    return era5var[varname]

def retrieve_era5_month(month,varid):
    varlist = []
    for year in era5yrs:
        var = retrieve_era5(year,varid)
        varmonth = var.sel(time="%s-%s"%(year,month))
        varlist.append(varmonth)
    return xr.concat(varlist, "time")

#climatological mean computation
def climat_mean(month,varid):
    """Compute the climatological mean of a variable specified by 'varid', for a specific month
        - month : str, eg. '05' for May
        - varid : str, eg. "128_130_t" for temperature
    """    
    return retrieve_era5_month(month,varid).mean("time")


def region_mean2D(variable,mask,box):
    """Given a 2D variable (lat, lon), compute a spatial mean within a specified region
    defined by a mask, inside a given box
        - variable = 3D xarray.dataarray. Dimensions must be named "latitude" and "longitude"
        - mask = 2D xarray.dataarray of 0s and 1s. Must have same grid and dimension names as 'variable'
        - box = list of four items, [lon1, lon2, lat1, lat2]
    """
    mask_box = mask.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2]))
    variable_box = variable.sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2]))
    maskedvar = variable_box*mask_box
    region_mean = maskedvar.fillna(0).sum(["latitude","longitude"])/ mask_box.sum(["latitude","longitude"])
    
    return region_mean


In [4]:
def CAPE_from_skewT(temp,rh):
    """Compute CAPE given a vertical profile of temperature and relative humidity"""
    p = np.array(temp.level)* units.hPa
    T = (np.array(temp)-273.15) * units.degC
    r = np.array(rh)/100
    Td = mpcalc.dewpoint_from_relative_humidity(T,r)
    
    # Calculate the profile of a parcel lifted from the surface
    prof = mpcalc.parcel_profile(p[::-1], T[-1], Td[-1]).to('degC')[::-1]
    
    #compute cape
    cape,_=mpcalc.cape_cin(p[::-1],T[::-1],Td[::-1],prof[::-1])

    return cape/units('J/kg')

# Masks, data

In [5]:
def slide_coord(mask):
    """Switch the longitude coord from (-180,180) to (0,360) 
    and change coordinate names from 'LAT1','LON1' to 'latitude' and 'longitude'"""
    mask0=mask.copy()
    mask0.coords['longitude'] = mask0.coords['LON1']%360
    mask1 = mask0.swap_dims({'LON1': 'longitude'}).rename({'LAT1': 'latitude'})

    #Sort the longitude values
    sort_inds = {"longitude": np.argsort(mask1["longitude"].values)}
    mask1 = mask1.isel(**sort_inds)
    return mask1

#masks = xr.open_dataset("/global/cscratch1/sd/qnicolas/autumnMonsoonData/winter_rainfall_masks.nc")
masks = xr.open_dataset("winter_rainfall_masks.nc")
trmm_nh_winter_mask = masks.TRMM_NH_WINTER_MASK.fillna(0.)
trmm_sh_winter_mask = masks.TRMM_SH_WINTER_MASK.fillna(0.)

REFERENCE_GRID = xr.open_dataset(ERA5M_PATH+"e5.moda.an.pl/e5.moda.an.pl.128_060_pv.ll025sc.1979010100_1979120100.nc").PV.sel(latitude=slice(50., -50.)).isel(time=0) #to get the era5 grid

trmm_nh_winter_mask1 = (slide_coord(trmm_nh_winter_mask).interp_like(REFERENCE_GRID) > 0)*1.
trmm_sh_winter_mask1 = (slide_coord(trmm_sh_winter_mask).interp_like(REFERENCE_GRID) > 0)*1.                

In [6]:
landmask = xr.open_dataset("/global/cfs/projectdirs/m3522/cmip6/ERA5/e5.oper.invariant/197901/e5.oper.invariant.128_172_lsm.ll025sc.1979010100_1979010100.nc").LSM.isel(time=0)
landmask50=landmask.interp_like(REFERENCE_GRID)


In [7]:
ghcn = xr.open_dataset("/global/cscratch1/sd/qnicolas/ghcn.precip.mon.total.nc",decode_times=False)
ghcn.coords['time'] = pd.to_datetime('1800-01-01') + pd.to_timedelta(np.array(ghcn.coords['time']),'d')
ghcn=ghcn.rename({'lat':'latitude','lon':'longitude'}).precip

In [8]:
# determine dry and wet years for each region,with ghcn
def timesel(year,month,targetmonth):
    return (year >= 1979) & (year < 2019) & (month == targetmonth)

ghcn_nov = ghcn.sel(time = timesel(ghcn['time.year'],ghcn['time.month'],11))
ghcn_may = ghcn.sel(time = timesel(ghcn['time.year'],ghcn['time.month'],5))

nwettest = int((2015-1979)/3);ndriest = nwettest
drywetyearsNH = []
for i,box in enumerate(boxNH1):
    ghcn_nov_box = np.array([region_mean2D(ghcn_nov.sel(latitude=slice(50., -50.),time=y),
                                           trmm_nh_winter_mask1.interp_like(ghcn_nov),box) for y in ghcn_nov.time])
    drywetyearsNH.append({"dry" : ghcn_nov.time[np.argpartition(ghcn_nov_box, ndriest)[:ndriest]], "wet" : ghcn_nov.time[np.argpartition(-ghcn_nov_box, nwettest)[:nwettest]]})

drywetyearsSH = []
for i,box in enumerate(boxSH1):
    ghcn_may_box = np.array([region_mean2D(ghcn_may.sel(latitude=slice(50., -50.),time=y),
                                           trmm_sh_winter_mask1.interp_like(ghcn_may),box) for y in ghcn_may.time])
    drywetyearsSH.append({"dry" : ghcn_may.time[np.argpartition(ghcn_may_box, ndriest)[:ndriest]], "wet" : ghcn_may.time[np.argpartition(-ghcn_may_box, nwettest)[:nwettest]]})
    

In [11]:
t=time.time()
all_tt_nov = retrieve_era5_month("11",'128_130_t' );all_tt_may = retrieve_era5_month("05",'128_130_t' )
all_rh_nov = retrieve_era5_month("11",'128_157_r' );all_rh_may = retrieve_era5_month("05",'128_157_r' )
print(time.time()-t)#takes ~6min

374.509877204895


In [18]:
all_cape_nov = retrieve_era5_month("11",'128_059_cape' );all_cape_may = retrieve_era5_month("05",'128_059_cape' )

In [12]:
mean_tt_nov = all_tt_nov.mean("time");mean_tt_may = all_tt_may.mean("time")
mean_rh_nov = all_rh_nov.mean("time");mean_rh_may = all_rh_may.mean("time")
mean_cape_nov = all_cape_nov.mean("time");mean_cape_may = all_cape_may.mean("time")

In [15]:
t=time.time()
ttNH = []; rhNH = []; capeNH = [] #Global mean quantities for the dry and wet years corresponding to each region
for i,box in enumerate(boxNH1):
    ttNH.append(  {"dry" : all_tt_nov.sel(time=drywetyearsNH[i]["dry"]).mean("time")  , "wet" : all_tt_nov.sel(time=drywetyearsNH[i]["wet"]).mean("time")  })
    rhNH.append(  {"dry" : all_rh_nov.sel(time=drywetyearsNH[i]["dry"]).mean("time")  , "wet" : all_rh_nov.sel(time=drywetyearsNH[i]["wet"]).mean("time")  }) 
    capeNH.append({"dry" : all_cape_nov.sel(time=drywetyearsNH[i]["dry"]).mean("time"), "wet" : all_cape_nov.sel(time=drywetyearsNH[i]["wet"]).mean("time")}) 

print(time.time()-t)

150.99146437644958


In [16]:
t=time.time()
ttSH = []; rhSH = []; capeSH = []
for i,box in enumerate(boxSH1):
    ttSH.append(  {"dry" : all_tt_may.sel(time=drywetyearsSH[i]["dry"]).mean("time")  , "wet" : all_tt_may.sel(time=drywetyearsSH[i]["wet"]).mean("time")  })
    rhSH.append(  {"dry" : all_rh_may.sel(time=drywetyearsSH[i]["dry"]).mean("time")  , "wet" : all_rh_may.sel(time=drywetyearsSH[i]["wet"]).mean("time")  }) 
    capeSH.append({"dry" : all_cape_may.sel(time=drywetyearsSH[i]["dry"]).mean("time"), "wet" : all_cape_may.sel(time=drywetyearsSH[i]["wet"]).mean("time")}) 

print(time.time()-t)

51.15903902053833


# Summary

In [26]:
from tabulate import tabulate

print(' '*40+'\033[1m'+ "May (SH)/ November(NH) CAPE values in J/kg" + '\033[0m')

tab=[]

for i,box in enumerate(boxNH1):
    cape_era5_mean = region_mean2D(mean_cape_nov, trmm_nh_winter_mask1, box)
    cape_era5_dry =  region_mean2D(capeNH[i]["dry"], trmm_nh_winter_mask1, box)
    cape_era5_wet =  region_mean2D(capeNH[i]["wet"], trmm_nh_winter_mask1, box)
    cape_skewT_mean = CAPE_from_skewT(region_mean2D(mean_tt_nov, trmm_nh_winter_mask1, box),region_mean2D(mean_rh_nov, trmm_nh_winter_mask1, box))
    
    tab.append([namesNH[i], "%.2f"%cape_era5_mean, "%.2f"%cape_era5_dry, "%.2f"%cape_era5_wet, "%.2f"%cape_skewT_mean])

for i,box in enumerate(boxSH1):
    cape_era5_mean = region_mean2D(mean_cape_may, trmm_sh_winter_mask1, box)
    cape_era5_dry =  region_mean2D(capeSH[i]["dry"], trmm_sh_winter_mask1, box)
    cape_era5_wet =  region_mean2D(capeSH[i]["wet"], trmm_sh_winter_mask1, box)
    cape_skewT_mean = CAPE_from_skewT(region_mean2D(mean_tt_may, trmm_sh_winter_mask1, box),region_mean2D(mean_rh_may, trmm_sh_winter_mask1, box))
    
    tab.append([namesSH[i], "%.2f"%cape_era5_mean, "%.2f"%cape_era5_dry, "%.2f"%cape_era5_wet, "%.2f"%cape_skewT_mean])

tab.sort(key=lambda l : float(l[1]))
    
print(tabulate(tab, headers=['Region'+' '*17, 'ERA5, mean 1979-2018', 'ERA5, dry yrs','ERA5, wet years','CAPE from mean sounding 1979-2018']))

    

                                        May (SH)/ November(NH) CAPE values in J/kg
Region                       ERA5, mean 1979-2018    ERA5, dry yrs    ERA5, wet years    CAPE from mean sounding 1979-2018
-------------------------  ----------------------  ---------------  -----------------  -----------------------------------
NorthEast Brazil                           331.71           320.61             336.16                               552.47
Tanzania                                   413.19           357.76             477.88                               576.88
Vietnam / South China Sea                  444.48           446.45             433.87                               347.01
South India / Sri Lanka                    454.2            445.1              474.89                               726.14
Malaysia                                   605.66           623.49             550.21                              1063.44
Philippines                                640.37       

# OND and AMJ

In [9]:
def selseason(month,monthinf,monthsup):
    return (np.mod(month - monthinf,12) < np.mod(monthsup - monthinf,12)) & (np.mod(monthsup - month,12) <= np.mod(monthsup - monthinf,12))

def retrieve_era5_season(monthinf,monthsup,varid):
    varlist = []
    for year in era5yrs:
        var = retrieve_era5(year,varid)
        varmonth = var.sel(time=selseason(var['time.month'],monthinf,monthsup)).mean('time')
        varlist.append(varmonth)
    return xr.concat(varlist, "time")


In [10]:
t=time.time()
all_tt_ond = retrieve_era5_season(10,1,'128_130_t' );all_tt_amj = retrieve_era5_season(4,7,'128_130_t' )
all_rh_ond = retrieve_era5_season(10,1,'128_157_r' );all_rh_amj = retrieve_era5_season(4,7,'128_157_r' )
all_cape_ond = retrieve_era5_season(10,1,'128_059_cape' );all_cape_amj = retrieve_era5_season(4,7,'128_059_cape' )
print(time.time()-t)#takes ~12min

732.9126734733582


In [24]:

all_tt_ond['time']   = all_cape_nov.time; all_tt_amj['time']   = all_cape_may.time
all_rh_ond['time']   = all_cape_nov.time; all_rh_amj['time']   = all_cape_may.time
all_cape_ond['time'] = all_cape_nov.time; all_cape_amj['time'] = all_cape_may.time


In [11]:
mean_tt_ond = all_tt_ond.mean("time");mean_tt_amj = all_tt_amj.mean("time")
mean_rh_ond = all_rh_ond.mean("time");mean_rh_amj = all_rh_amj.mean("time")
mean_cape_ond = all_cape_ond.mean("time");mean_cape_amj = all_cape_amj.mean("time")

In [22]:
t=time.time()
ttNH_season = []; rhNH_season = []; capeNH_season = [] #Global mean quantities for the dry and wet years corresponding to each region
for i,box in enumerate(boxNH1):
    ttNH_season.append(  {"dry" :   all_tt_ond.sel(time=np.in1d(all_tt_ond['time.year'],drywetyearsNH[i]["dry"]["time.year"])).mean("time")  , "wet" :    all_tt_ond.sel(time=np.in1d(all_tt_ond['time.year'],drywetyearsNH[i]["wet"]["time.year"])).mean("time")  })
    rhNH_season.append(  {"dry" :   all_rh_ond.sel(time=np.in1d(all_rh_ond['time.year'],drywetyearsNH[i]["dry"]["time.year"])).mean("time")  , "wet" :    all_rh_ond.sel(time=np.in1d(all_rh_ond['time.year'],drywetyearsNH[i]["wet"]["time.year"])).mean("time")  }) 
    capeNH_season.append({"dry" : all_cape_ond.sel(time=np.in1d(all_cape_ond['time.year'],drywetyearsNH[i]["dry"]["time.year"])).mean("time"), "wet" :  all_cape_ond.sel(time=np.in1d(all_cape_ond['time.year'],drywetyearsNH[i]["wet"]["time.year"])).mean("time")}) 

print(time.time()-t)

114.52869939804077


In [25]:
t=time.time()
ttSH_season = []; rhSH_season = []; capeSH_season = [] #Global mean quantities for the dry and wet years corresponding to each region
for i,box in enumerate(boxSH1):
    ttSH_season.append(  {"dry" :   all_tt_amj.sel(time=np.in1d(all_tt_amj['time.year'],drywetyearsSH[i]["dry"]["time.year"])).mean("time")  , "wet" :    all_tt_amj.sel(time=np.in1d(all_tt_amj['time.year'],drywetyearsSH[i]["wet"]["time.year"])).mean("time")  })
    rhSH_season.append(  {"dry" :   all_rh_amj.sel(time=np.in1d(all_rh_amj['time.year'],drywetyearsSH[i]["dry"]["time.year"])).mean("time")  , "wet" :    all_rh_amj.sel(time=np.in1d(all_rh_amj['time.year'],drywetyearsSH[i]["wet"]["time.year"])).mean("time")  }) 
    capeSH_season.append({"dry" : all_cape_amj.sel(time=np.in1d(all_cape_amj['time.year'],drywetyearsSH[i]["dry"]["time.year"])).mean("time"), "wet" :  all_cape_amj.sel(time=np.in1d(all_cape_amj['time.year'],drywetyearsSH[i]["wet"]["time.year"])).mean("time")}) 

print(time.time()-t)

43.483721017837524


In [27]:
from tabulate import tabulate

print(' '*40+'\033[1m'+ "AMJ (SH)/ OND(NH) CAPE values in J/kg" + '\033[0m')

tab=[]

for i,box in enumerate(boxNH1):
    cape_era5_mean = region_mean2D(mean_cape_ond, trmm_nh_winter_mask1, box)
    cape_era5_dry =  region_mean2D(capeNH_season[i]["dry"], trmm_nh_winter_mask1, box)
    cape_era5_wet =  region_mean2D(capeNH_season[i]["wet"], trmm_nh_winter_mask1, box)
    cape_skewT_mean = CAPE_from_skewT(region_mean2D(mean_tt_ond, trmm_nh_winter_mask1, box),region_mean2D(mean_rh_ond, trmm_nh_winter_mask1, box))
    
    tab.append([namesNH[i], "%.2f"%cape_era5_mean, "%.2f"%cape_era5_dry, "%.2f"%cape_era5_wet, "%.2f"%cape_skewT_mean])

for i,box in enumerate(boxSH1):
    cape_era5_mean = region_mean2D(mean_cape_amj, trmm_sh_winter_mask1, box)
    cape_era5_dry =  region_mean2D(capeSH_season[i]["dry"], trmm_sh_winter_mask1, box)
    cape_era5_wet =  region_mean2D(capeSH_season[i]["wet"], trmm_sh_winter_mask1, box)
    cape_skewT_mean = CAPE_from_skewT(region_mean2D(mean_tt_amj, trmm_sh_winter_mask1, box),region_mean2D(mean_rh_amj, trmm_sh_winter_mask1, box))
    
    tab.append([namesSH[i], "%.2f"%cape_era5_mean, "%.2f"%cape_era5_dry, "%.2f"%cape_era5_wet, "%.2f"%cape_skewT_mean])

tab.sort(key=lambda l : float(l[1]))

print(tabulate(tab, headers=['Region'+' '*17, 'ERA5, mean 1979-2018', 'ERA5, dry yrs','ERA5, wet years','CAPE from mean sounding 1979-2018']))

    

                                        AMJ (SH)/ OND(NH) CAPE values in J/kg
Region                       ERA5, mean 1979-2018    ERA5, dry yrs    ERA5, wet years    CAPE from mean sounding 1979-2018
-------------------------  ----------------------  ---------------  -----------------  -----------------------------------
NorthEast Brazil                           331.21           315.84             345.83                               509.1
Vietnam / South China Sea                  425.04           415.47             416.19                               219.58
Tanzania                                   431.19           397.47             475.19                               469.68
South India / Sri Lanka                    496.72           490.9              512.51                               680.44
Malaysia                                   598.21           593.07             576.46                               974.54
Philippines                                599.64           58